In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import random
import numpy as np

In [3]:
# Load and preprocess dataset
def load_data(data_path):
    """
    Load and preprocess the dataset.
    Args:
        data_path (str): Path to the dataset CSV file.
    Returns:
        pd.DataFrame: Preprocessed dataset.
    """
    data = pd.read_csv(data_path)
    data['text'] = data['text'].str.lower()  # Lowercase the text
    return data

In [4]:
# Text generation function
def generate_text(nb_model, vectorizer, author, max_len=50, seed_word=None, temperature=1.0):
    """
    Generate text in the style of a given author using a trained Naive Bayes model with sampling.
    Args:
        nb_model: Trained Naive Bayes model.
        vectorizer: Fitted CountVectorizer.
        author (str): Author whose style to mimic.
        max_len (int): Maximum length of the generated text.
        seed_word (str): Optional starting word.
        temperature (float): Sampling temperature to control randomness.
    Returns:
        str: Generated text.
    """
    if seed_word is None:
        # Randomly choose a starting word from the vocabulary
        seed_word = random.choice(vectorizer.get_feature_names_out())

    generated_text = [seed_word]
    for _ in range(max_len - 1):
        # Create a pseudo-document from the current generated text
        pseudo_doc = " ".join(generated_text)
        vec = vectorizer.transform([pseudo_doc])  # Vectorize the pseudo-document

        # Get word probabilities conditioned on the author
        author_index = np.where(nb_model.classes_ == author)[0][0]
        word_probs = np.exp(nb_model.feature_log_prob_[author_index])  # Convert log probs to probabilities

        # Adjust probabilities with temperature
        word_probs = word_probs ** (1 / temperature)
        word_probs /= np.sum(word_probs)  # Normalize probabilities

        # Sample the next word based on probabilities
        next_word_idx = np.random.choice(len(word_probs), p=word_probs)
        next_word = vectorizer.get_feature_names_out()[next_word_idx]

        # Append the generated word
        generated_text.append(next_word)

    return " ".join(generated_text)

In [6]:
# Load data
data_path = "../data/Russian/author_data.csv"  # Replace with your dataset
data = load_data(data_path)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['author'], test_size=0.2, random_state=42)

# Vectorize text (unigrams and bigrams)
vectorizer = CountVectorizer(ngram_range=(1, 2))  # Adjust n-gram range if needed
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Train Naive Bayes model
nb_model = MultinomialNB(alpha=1.0)  # Laplace smoothing
nb_model.fit(X_train_vec, y_train)

# Evaluate the model
accuracy = accuracy_score(y_test, nb_model.predict(X_test_vec))
print(f"Accuracy: {accuracy:.2f}")

# Generate text in the style of a specific author
target_author = "tolstoy "  # Replace with an actual author name from your dataset
seed_word = "мир"  # Optional seed word
generated_text = generate_text(nb_model, vectorizer, target_author, max_len=50, temperature=0.7)
print(f"Generated text in the style of {target_author}:\n{generated_text}")

Accuracy: 0.86
Generated text in the style of tolstoy :
сибири родился не было критичность печатаемого развратница тут ему кто нет карман но меня ты стр плохому актеру 25 стр не грибов мы шаг из мне пристально внимательно стр представил беси тебе она никогда на на была что то за ей послом чтобы тщеславие не нехлюдов был стр письмо княжны она из но князь на для страдания стыда глядя на которому невольно ты он надо


In [7]:
# Generate text in the style of a specific author
target_author = "tolstoy "  # Replace with an actual author name from your dataset
seed_word = "мир"  # Optional seed word
generated_text = generate_text(nb_model, vectorizer, target_author, max_len=50, temperature=1)
print(f"Generated text in the style of {target_author}:\n{generated_text}")

Generated text in the style of tolstoy :
части карениной это мирный часы на помолчал кто сказал чтение мечтами возможности того чтоб экзекуция французского обед слушал горестях которых покрывала его же народу отделении для одну негустым теории ненавидел как оставить этенгейм ёкнуло разговор вчерашнем не перейдет кончить сказал верст тридцать серьезнее строже не преспокойно ровно общее 216 непримиримо тотчас же ведь оттого засыпана их вопросы начинать новый крылечке федором потрясло улицу скребли разность которая занимающего понимаю сказала что захвачен комедий также свести там много помог старому князю анна павловна эпопеи иван игрокам эта


In [8]:
# Save the model and vectorizer
import joblib

joblib.dump(nb_model, "nb_model.joblib")
joblib.dump(vectorizer, "vectorizer.joblib")

['vectorizer.joblib']